In [1]:
MONK_TASK = 1
USER = 'Nunzio'

In [2]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = '/content/drive/Shareddrives/Project_ML_23/' + USER + '/machine-learning-project'
    sys.path.insert(0,BASE_PATH)
    N_JOBS = 1
    sys.path.insert(0,BASE_PATH + '/src/utils')
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'monk',f'monks-{MONK_TASK}.train')
    TEST_DATA = os.path.join(BASE_PATH, 'datasets', 'monk',f'monks-{MONK_TASK}.test')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'monk',  f'task-{MONK_TASK}', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'monk', f'task-{MONK_TASK}')
else :
    TRAIN_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.train')
    TEST_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.test')
    N_JOBS = -1
    IMAGES_FOLDER = os.path.join('..', '..', '..', 'images', 'monk', f'task-{MONK_TASK}', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', '..', 'trained_models', 'monk', f'task-{MONK_TASK}')

In [3]:
if (colab := 'google.colab' in sys.modules):
    sys.path.append(BASE_PATH + '/src/utils')
else:
    sys.path.append('../../utils')

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')

from utils import save_plot, set_random_state
from NN import MonkNeuralNetwork

set_random_state(42)

In [4]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 8))

df_train = pd.read_csv(TRAIN_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_test = pd.read_csv(TEST_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_train.head()

,1,2,3,4,5,6,7
0,1,1,1,1,1,3,1
1,1,1,1,1,1,3,2
2,1,1,1,1,3,2,1
3,1,1,1,1,3,3,2
4,1,1,1,2,1,2,1


In [5]:
features = ['feature_' + str(i) for i in range(1, 7)]

# Rename columns
new_column_names = ['class'] + features

df_train.columns = new_column_names
df_test.columns = new_column_names

df_train.head()

,class,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,1,1,1,1,1,3,1
1,1,1,1,1,1,3,2
2,1,1,1,1,3,2,1
3,1,1,1,1,3,3,2
4,1,1,1,2,1,2,1


In [6]:
df_train_encoded = pd.get_dummies(df_train, columns=features)
df_test_encoded = pd.get_dummies(df_test, columns=features)

df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join='inner', axis=1)

df_train_encoded.head()

,class,feature_1_1,feature_1_2,feature_1_3,feature_2_1,feature_2_2,feature_2_3,feature_3_1,feature_3_2,feature_4_1,feature_4_2,feature_4_3,feature_5_1,feature_5_2,feature_5_3,feature_5_4,feature_6_1,feature_6_2
0,1,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,True,False
1,1,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True
2,1,True,False,False,True,False,False,True,False,False,False,True,False,True,False,False,True,False
3,1,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,True
4,1,True,False,False,True,False,False,False,True,True,False,False,False,True,False,False,True,False


In [7]:
features = df_train_encoded.columns.difference(['class'])

X_train = df_train_encoded[features].to_numpy()
y_train = df_train_encoded['class'].to_numpy()

X_test = df_test_encoded[features].to_numpy()
y_test = df_test_encoded['class'].to_numpy()

# Create model

In [8]:
model = MonkNeuralNetwork(input_dim=X_train.shape[1], verbose=0)

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'architecture': [
        (8,), (8, 8), (8, 8, 8), (8, 8, 8, 8)
    ],
    'optimizer': ['sgd'],
    'activation': ['relu'],
    'learning_rate': [0.1, 0.3, 0.5],
    'momentum': [0.6, 0.8, 0.9],
    'lambda_value': [0.01],
    'epochs': [200],
    'batch_size': [8],
    'patience': [6]
}
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    n_jobs=N_JOBS,
    verbose=2,
    scoring='accuracy'
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


2024-01-19 15:40:02.845382: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-19 15:40:02.845560: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-19 15:40:02.845592: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-19 15:40:02.845978: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-19 15:40:02.846361: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-01-19 15:40:03.146258: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-19 15:40:03.146568: I metal_plugin/src/device/metal_device.cc:296] system

[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.8, optimizer=sgd, patience=6; total time=   7.7s
[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.8, optimizer=sgd, patience=6; total time=   8.5s


/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/Users/nun

[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.6, optimizer=sgd, patience=6; total time=   8.7s


...

[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.8, optimizer=sgd, patience=6; total time=  17.0s
[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.9, optimizer=sgd, patience=6; total time=   6.2s


/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/Users/nun

[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.6, optimizer=sgd, patience=6; total time=  18.2s
[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.9, optimizer=sgd, patience=6; total time=   6.8s
[CV] END activation=relu, architecture=(8,), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.9, optimizer=sgd, patience=6; total time=  10.5s
[CV] END activation=relu, architecture=(8, 8), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.8, optimizer=sgd, patience=6; total time=   8.9s
[CV] END activation=relu, architecture=(8, 8), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.6, optimizer=sgd, patience=6; total time=  14.1s
[CV] END activation=relu, architecture=(8, 8), batch_size=8, epochs=200, lambda_value=0.01, learning_rate=0.1, momentum=0.8, optimizer=sgd, patience=6;

/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/sklearn/model_selection/_search.py:979: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(
2024-01-19 15:40:45.097467: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-19 15:40:45.097536: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-19 15:40:45.097551: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-19 15:40:45.097766: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-19 15:40:45.097788: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bu

GridSearchCV(cv=5, estimator=MonkNeuralNetwork(verbose=0), n_jobs=-1,
             param_grid={'activation': ['relu'], 'architecture': [(8,), (8, 8)],
                         'batch_size': [8], 'epochs': [200],
                         'lambda_value': [0.01], 'learning_rate': [0.1],
                         'momentum': [0.6, 0.8, 0.9], 'optimizer': ['sgd'],
                         'patience': [6]},
             scoring='accuracy', verbose=2)

In [11]:
results = grid_search.cv_results_

# Converti i risultati in un DataFrame per una migliore visualizzazione
results_df = pd.DataFrame(results)
csv_filename = f"Monk_NN_{MONK_TASK}.csv"
results_df.to_csv(csv_filename, index=False)

In [32]:

from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)
train_scores, test_scores = [], []

best_params = grid_search.best_params_


# Cross-validation con i migliori parametri
for train_index, test_index in cv.split(X_train):
    X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]
    
    best_model = MonkNeuralNetwork(input_dim=X_train_1.shape[1], verbose=0, **best_params)
    best_model.fit(X_train_1, y_train_1, validation_data=(X_test_1, y_test_1))
    
    #df = best_model.history.history
    train_scores.append(best_model.history.history)

In [22]:
train_scores_df = pd.DataFrame(train_scores)

In [ ]:
# TODO : 
# plottare risultati di ogni fold, plottando la media(o il best) dei fold e la varianza per ogni fold
# hold out test set, usarlo per il model assestment 
# alla fine fare retrain di tutto compreso hold out test set prima di fare predict sui risultati di micheli 

In [41]:
train_scores_df['val_loss']

[[0.38247257471084595,
  0.3527109622955322,
  0.33312568068504333,
  0.3153890371322632,
  0.29734981060028076,
  0.2872540056705475,
  0.2744085192680359,
  0.26684626936912537,
  0.25725963711738586,
  0.2530861794948578,
  0.2530364990234375,
  0.24381113052368164,
  0.23737452924251556,
  0.23881256580352783,
  0.23551641404628754,
  0.23358725011348724,
  0.22846242785453796,
  0.22916603088378906,
  0.2255028784275055,
  0.22855272889137268,
  0.22006756067276,
  0.21259425580501556,
  0.21475425362586975,
  0.20953451097011566,
  0.2137918323278427,
  0.20895114541053772,
  0.2143259048461914,
  0.20448806881904602,
  0.21443703770637512,
  0.2092023342847824,
  0.20617450773715973,
  0.2056771218776703,
  0.20387975871562958,
  0.2005126178264618,
  0.20154818892478943,
  0.2100171446800232,
  0.20151197910308838,
  0.19864588975906372,
  0.19892454147338867,
  0.19942227005958557,
  0.20027942955493927,
  0.19937512278556824,
  0.2019125521183014,
  0.19918084144592285,
  0.1

In [34]:

# Create dataframes for train_scores and test_scores
train_scores_filename = f"Monk_NN_{MONK_TASK}_train_scores.csv"

# Save dataframes to CSV files
train_scores_df.to_csv(train_scores_filename, index=False)


In [ ]:
final_model = grid_search.best_estimator_


In [ ]:
final_model.history.history

In [35]:
from sklearn.metrics import accuracy_score, classification_report

# Test set accuracy hold-out

final_model = grid_search.best_estimator_
y_pred = final_model.predict(X_test)

print('Best parameters: ', grid_search.best_params_)
print('Best accuracy: ', grid_search.best_score_)
print('Test set accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

1/1 [==============================] - 0s 106ms/step
Best parameters:  {'activation': 'relu', 'architecture': (8,), 'batch_size': 8, 'epochs': 200, 'lambda_value': 0.01, 'learning_rate': 0.1, 'momentum': 0.6, 'optimizer': 'sgd', 'patience': 6}
Best accuracy:  nan
Test set accuracy:  0.9230769230769231
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.88      0.93         8

    accuracy                           0.92        13
   macro avg       0.92      0.94      0.92        13
weighted avg       0.94      0.92      0.92        13



# Learning curve

In [37]:
train_scores_df[['loss', 'val_loss']].plot(
    figsize=(8, 5),
    xlim=[0, 200],
    ylim=[0, 2],
    grid=True,
    xlabel='Epoch',
    style=['r--', 'b--'],
    title='Training and Validation Loss'
)
plt.show()

KeyError: 0

In [ ]:
history_df[['accuracy', 'val_accuracy']].plot(
    figsize=(8, 5),
    xlim=[0, 200],
    ylim=[0, 1],
    grid=True,
    xlabel='Epoch',
    style=['r--.', 'b--'],
    title='Training and Validation Accuracy'
)
plt.show()

# Save model

In [ ]:
from joblib import dump

model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)